<a href="https://colab.research.google.com/github/banyujati/TNJ-banyu/blob/main/06_huggingface_api_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with External APIs - Hugging Face

## Session 6: API Basics for AI

Learn to use external AI APIs to leverage pre-trained models without building them yourself!

### Learning Objectives:
- Make HTTP requests with Python
- Understand JSON responses
- Use Hugging Face Inference API
- Handle API authentication


## 1. Making HTTP Requests

APIs communicate over HTTP. Let's learn the basics!


In [1]:
import requests
import json

# Simple GET request
response = requests.get('https://api.github.com')
print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    data = response.json()
    print(f"Response keys: {list(data.keys())[:5]}...")
    print("✅ Request successful!")


Status Code: 200
Response keys: ['current_user_url', 'current_user_authorizations_html_url', 'authorizations_url', 'code_search_url', 'commit_search_url']...
✅ Request successful!


## 2. Understanding JSON

JSON (JavaScript Object Notation) is how APIs send and receive data.


In [ ]:
# JSON example - this is how APIs return data
api_response = {
    "prediction": "cat",
    "confidence": 0.92,
    "model": "image-classifier"
}

# Convert Python dict to JSON string
json_string = json.dumps(api_response)
print(f"JSON string: {json_string}")

# Convert JSON string back to Python dict
parsed = json.loads(json_string)
print(f"\nParsed: {parsed}")
print(f"Prediction: {parsed['prediction']}")
print(f"Confidence: {parsed['confidence']}")


## 3. Hugging Face Inference API

Hugging Face provides free access to many pre-trained AI models!


In [ ]:
# Hugging Face Inference API endpoint
# Note: Hugging Face now requires an API token for most models
# Get a free token at: https://huggingface.co/settings/tokens
#
# Steps to get your token:
# 1. Create a free account at https://huggingface.co/join
# 2. Go to https://huggingface.co/settings/tokens
# 3. Create a new token with these permissions:
#    - "Read" access (for accessing models)
#    - "Make calls to Inference Providers" (IMPORTANT for API calls)
# 4. Copy the token and use it below

# Correct API endpoint format (using new router endpoint)
# The endpoint format is: https://router.huggingface.co/hf-inference/models/{organization}/{model_name}
# Example: https://router.huggingface.co/hf-inference/models/distilbert/distilbert-base-uncased-finetuned-sst-2-english
API_URL = "https://router.huggingface.co/hf-inference/models"

# Example: Text classification model
# Note: Model path should include organization/model_name format
# You can find the exact path on the model's page on Hugging Face Hub
#
# Option 1: Original sentiment analysis model (with organization prefix)
model_path = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

# Option 2: If above gives 404, try this simpler model (uncomment to use):
# model_path = "gpt2"  # Simple text generation model - always available
# data = {"inputs": "The weather today is"}

# Option 3: Or try this zero-shot classification model:
# model_path = "facebook/bart-large-mnli"
# data = {
#     "inputs": "I love this course!",
#     "parameters": {"candidate_labels": ["positive", "negative", "neutral"]}
# }

# IMPORTANT: Replace with your actual token (or set as environment variable)
# For security, consider using: os.getenv("HF_API_TOKEN") or os.environ.get("HF_API_TOKEN")
HF_TOKEN = "YOUR_TOKEN_HER"  # Replace with your token

# Check if token is set
if HF_TOKEN == "YOUR_TOKEN_HERE":
    print("⚠️ WARNING: Please replace 'YOUR_TOKEN_HERE' with your actual Hugging Face API token!")
    print("Get your token at: https://huggingface.co/settings/tokens")
else:
    print(f"✅ Token found (first 10 chars: {HF_TOKEN[:10]}...)")

# Prepare headers with authentication
# Note: Content-Type is usually automatically set by requests.post with json parameter
headers = {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/json"
}

# Prepare data for text classification
data = {
    "inputs": "I love this course!"
}

# Make API request
try:
    # Construct the full URL: https://router.huggingface.co/hf-inference/models/{organization}/{model_name}
    url = f"{API_URL}/{model_path}"

    print(f"\n📡 Making request to: {url}")
    print(f"📝 Model: {model_path}")
    print(f"📤 Input: {data['inputs']}\n")

    response = requests.post(
        url,
        json=data,
        headers=headers,
        timeout=30
    )

    print(f"📥 Response Status: {response.status_code}")

    if response.status_code == 200:
        result = response.json()
        print(f"✅ API Response:")
        print(json.dumps(result, indent=2))
    elif response.status_code == 401:
        print(f"⚠️ Status Code: {response.status_code} - Unauthorized")
        print("Please check your API token:")
        print("1. Make sure you've replaced 'YOUR_TOKEN_HERE' with your actual token")
        print("2. Get your token at: https://huggingface.co/settings/tokens")
        print("3. Ensure the token has 'read' permissions")
        print(f"4. Current token (first 10 chars): {HF_TOKEN[:10]}...")
    elif response.status_code == 404:
        print(f"⚠️ Status Code: {response.status_code} - Not Found")
        print("\nPossible issues:")
        print(f"1. Model path might be incorrect: '{model_path}'")
        print("2. Check if the model exists at: https://huggingface.co/models")
        print("3. Try visiting: https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english")
        print("4. Make sure the model path includes organization/model_name format")
        print("5. Some models may require accepting terms and conditions first")
        print(f"\nFull URL attempted: {url}")
        print(f"Response: {response.text[:500]}")
    elif response.status_code == 503:
        print(f"⚠️ Status Code: {response.status_code} - Model is loading")
        print("The model is starting up. Please wait 10-30 seconds and try again.")
        try:
            error_info = response.json()
            print(f"Response: {json.dumps(error_info, indent=2)}")
        except:
            print(f"Response: {response.text[:200]}")
    else:
        print(f"⚠️ Status Code: {response.status_code}")
        try:
            error_info = response.json()
            print(f"Response: {json.dumps(error_info, indent=2)}")
        except:
            print(f"Response: {response.text[:500]}")
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("Note: Check your internet connection and API token.")


## 4. Error Handling for APIs

APIs can fail - always handle errors gracefully!


In [ ]:
def call_api_safely(url, data, headers=None):
    """Safely call an API with error handling"""
    try:
        response = requests.post(url, json=data, headers=headers, timeout=10)
        response.raise_for_status()  # Raises exception for bad status codes
        return {"success": True, "data": response.json()}
    except requests.exceptions.Timeout:
        return {"success": False, "error": "Request timed out"}
    except requests.exceptions.HTTPError as e:
        return {"success": False, "error": f"HTTP error: {e.response.status_code}"}
    except requests.exceptions.RequestException as e:
        return {"success": False, "error": f"Request failed: {str(e)}"}
    except json.JSONDecodeError:
        return {"success": False, "error": "Invalid JSON response"}

# Test the function
result = call_api_safely("https://httpbin.org/post", {"test": "data"})
print(f"Test result: {result}")


## Summary

- ✅ Making HTTP requests with requests library
- ✅ Understanding and parsing JSON responses
- ✅ Using Hugging Face Inference API
- ✅ Error handling for API calls

In the next session, you'll use these skills to build a complete AI application!

---

**Session Complete!** 🎉
